# Fetch aircraft metadata (icao24 -> aircraft type)

Goal: build a lookup table that maps `icao24` to aircraft metadata (e.g., typecode / model).

- Uses the same date range as `01_fetch_flight_data_opensky_trino.ipynb`
- Respects OpenSky access guidelines by batching queries and sleeping between requests

Output: `data/processed/flight_data/states_europe/2025-01-13_2025-01-14/opensky_aircraft_metadata.parquet`


## 1) Config + project paths


In [1]:
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().parent
DATE_RANGE = "2025-01-13_2025-01-14"

RAW_STATES_DIR = ROOT / "data" / "raw" / "flight_data" / "states_europe" / DATE_RANGE
OUT_DIR = ROOT / "data" / "processed" / "flight_data" / "states_europe" / DATE_RANGE
OUT_PATH = OUT_DIR / "opensky_aircraft_metadata.parquet"

print("RAW_STATES_DIR:", RAW_STATES_DIR)
print("OUT_PATH:", OUT_PATH)


RAW_STATES_DIR: c:\Users\HiWi\Desktop\Terril\01_nextcloud\Germany\DATA SCIENCE\Semesters\05\02 Sustainability in aviation\03 contrail-mvp\data\raw\flight_data\states_europe\2025-01-13_2025-01-14
OUT_PATH: c:\Users\HiWi\Desktop\Terril\01_nextcloud\Germany\DATA SCIENCE\Semesters\05\02 Sustainability in aviation\03 contrail-mvp\data\processed\flight_data\states_europe\2025-01-13_2025-01-14\opensky_aircraft_metadata.parquet


## 2) Collect unique `icao24` observed in the selected time window


In [2]:
import pyarrow.parquet as pq

files = sorted(RAW_STATES_DIR.glob("*.parquet"))
if not files:
    raise FileNotFoundError(f"No parquet files found in {RAW_STATES_DIR}")

icao = set()
for f in files:
    # Read only icao24 column (fast)
    col = pq.read_table(f, columns=["icao24"]).column("icao24")
    icao.update(x.as_py() for x in col if x is not None)

icao24_list = sorted({str(x).lower() for x in icao if str(x).strip()})
print("Unique icao24:", f"{len(icao24_list):,}")
icao24_list[:10]


Unique icao24: 6,759


['000001',
 '008ba5',
 '00a2e4',
 '0100a7',
 '0100db',
 '0100dc',
 '0100e0',
 '0100e7',
 '010109',
 '01010a']

## 3) Connect to OpenSky Trino


In [3]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication, ConsoleRedirectHandler

# Use the same connection pattern as in 01_fetch_flight_data_opensky_trino.ipynb
# OpenSky requires a lowercase username here.
TRINO_USER = "terriljoel98"  # TODO: set to your OpenSky username (lowercase)

conn = connect(
    host="trino.opensky-network.org",
    port=443,
    http_scheme="https",
    user=TRINO_USER,
    auth=OAuth2Authentication(
        redirect_auth_url_handler=ConsoleRedirectHandler()
    ),
    catalog="minio",
    schema="osky",
)

print("Connected.")


Connected.


## Explore available OpenSky Trino tables

This project Trino schema (`minio.osky`) does not necessarily contain an aircraft metadata table (aircraft type/model).
First, list available tables and inspect column schemas with `DESCRIBE` before writing any fetch logic.


In [17]:
table_list = []
for key, value in tables.to_dict()['Table'].items():
    print(f"{value}")
    table_list.append(value)

acas_data4
adsc
allcall_replies_data4
flarm_raw
flights_data4
flights_data5
identification_data4
operational_status_data4
position_data4
rollcall_replies_data4
state_vectors_data4
velocity_data4


In [20]:
import pandas as pd

# List tables
tables = pd.read_sql("SHOW TABLES FROM minio.osky", conn)
col = tables.columns[0]
print("Tables in minio.osky:")
print(tables.sort_values(col).to_string(index=False))

# Helper: describe any table
def describe_table(table_name: str) -> pd.DataFrame:
    df = pd.read_sql(f"DESCRIBE {table_name}", conn)
    return df

# Example: inspect identification_data4
for table in table_list:
    print("\nTable:", table)
    print(describe_table("minio.osky.{}".format(table)))


C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\2953346535.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql("SHOW TABLES FROM minio.osky", conn)


Tables in minio.osky:
                   Table
              acas_data4
                    adsc
   allcall_replies_data4
               flarm_raw
           flights_data4
           flights_data5
    identification_data4
operational_status_data4
          position_data4
  rollcall_replies_data4
     state_vectors_data4
          velocity_data4

Table: acas_data4


C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\2953346535.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"DESCRIBE {table_name}", conn)


                          Column  \
0                        sensors   
1                         rawmsg   
2                        mintime   
3                        maxtime   
4                       msgcount   
5                         icao24   
6                   islongformat   
7                     isairborne   
8         hascrosslinkcapability   
9               sensitivitylevel   
10              replyinformation   
11                      altitude   
12                   hasvalidrac   
13    activeresolutionadvisories   
14  resolutionadvisorycomplement   
15                   nopassbelow   
16                   nopassabove   
17                    noturnleft   
18                   noturnright   
19                 hasterminated   
20            hasmultiplethreats   
21                          hour   

                                                 Type          Extra Comment  
0   array(row(serial integer, mintime double, maxt...                         
1            

C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\2953346535.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"DESCRIBE {table_name}", conn)


...

                  Column                                               Type  \
0                sensors  array(row(serial integer, mintime double, maxt...   
1                 rawmsg                                            varchar   
2                mintime                                             double   
3                maxtime                                             double   
4               msgcount                                             bigint   
5                 icao24                                            varchar   
6                message                                            varchar   
7                   isid                                            boolean   
8           flightstatus                                            tinyint   
9        downlinkrequest                                            tinyint   
10            utilitymsg                                            tinyint   
11        interrogatorid                            

C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\2953346535.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"DESCRIBE {table_name}", conn)


           Column            Type          Extra Comment
0            time         integer                       
1          icao24         varchar                       
2             lat          double                       
3             lon          double                       
4        velocity          double                       
5         heading          double                       
6        vertrate          double                       
7        callsign         varchar                       
8        onground         boolean                       
9           alert         boolean                       
10            spi         boolean                       
11         squawk         varchar                       
12   baroaltitude          double                       
13    geoaltitude          double                       
14  lastposupdate          double                       
15    lastcontact          double                       
16        serials  array(intege

C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\2953346535.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"DESCRIBE {table_name}", conn)


           Column                                               Type  \
0         sensors  array(row(serial integer, mintime double, maxt...   
1          rawmsg                                            varchar   
2         mintime                                             double   
3         maxtime                                             double   
4        msgcount                                             bigint   
5          icao24                                            varchar   
6      supersonic                                            boolean   
7    intentchange                                            boolean   
8   ifrcapability                                            boolean   
9             nac                                           smallint   
10     ewvelocity                                             double   
11     nsvelocity                                             double   
12           baro                                            boo

## 4) Discover aircraft metadata table

If you are unsure about the table name/columns, run this cell to list tables containing the word `aircraft`.


In [4]:
import pandas as pd

tables = pd.read_sql("SHOW TABLES FROM minio.osky", conn)
tables.columns = [c.lower() for c in tables.columns]
cand = tables[tables[tables.columns[0]].str.contains("aircraft", case=False, na=False)]
print(cand)


C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\3674217301.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql("SHOW TABLES FROM minio.osky", conn)


Open the following URL in browser for the external authentication:
https://trino.opensky-network.org/oauth2/token/initiate/a812bf5b62e6c69d560d194b59f3e5c435face485b0c82e5ec7cdc5676bf1de7
Empty DataFrame
Columns: [table]
Index: []


In [5]:
tables

,table
0,acas_data4
1,adsc
2,allcall_replies_data4
3,flarm_raw
4,flights_data4
5,flights_data5
6,identification_data4
7,operational_status_data4
8,position_data4
9,rollcall_replies_data4


## 5) Fetch aircraft metadata for observed icao24

Notes:
- Keep `chunk_size` moderate to avoid overly large SQL `IN (...)` clauses.
- Sleep between chunks to be polite to the service.
- Adjust `AIRCRAFT_TABLE` and selected columns to match what exists in your OpenSky schema.


In [7]:
import time
from tqdm.auto import tqdm
import pandas as pd

# Auto-discover an aircraft metadata table in OpenSky Trino.
# We prefer tables containing 'aircraft' and having an 'icao24' column.

def list_tables() -> list[str]:
    df = pd.read_sql("SHOW TABLES FROM minio.osky", conn)
    # Trino returns a single column; normalize to python list
    col = df.columns[0]
    return df[col].astype(str).tolist()

def describe_cols(table: str) -> set[str]:
    df = pd.read_sql(f"DESCRIBE {table}", conn)
    col = df.columns[0]
    return set(df[col].astype(str).str.lower().tolist())

tables = list_tables()
# Candidate tables containing 'aircraft' (sorted for reproducibility)
candidates = sorted([t for t in tables ])
if not candidates:
    raise RuntimeError("No tables containing 'aircraft' found in minio.osky. Run SHOW TABLES to inspect schema.")

# Prefer some common names first, then fall back to any candidate
preferred = [
    "aircraft_database",
    "aircraft",
    "aircraft_data",
    "aircraft_database4",
]
ordered = []
for name in preferred:
    ordered.extend([t for t in candidates if t.lower() == name])
ordered.extend([t for t in candidates if t not in ordered])

AIRCRAFT_TABLE = None
available_cols = None
for t in ordered:
    full = f"minio.osky.{t}" if not t.lower().startswith("minio.") else t
    try:
        cols = describe_cols(full)
    except Exception:
        continue
    if "icao24" in cols:
        AIRCRAFT_TABLE = full
        available_cols = cols
        break

if AIRCRAFT_TABLE is None:
    raise RuntimeError(f"Could not find an aircraft table with an icao24 column. Candidates tried: {ordered[:20]}")

print("Using AIRCRAFT_TABLE:", AIRCRAFT_TABLE)

# Candidate columns (keep only what you need); some may not exist depending on schema
CANDIDATE_COLS = [
    "icao24",
    "typecode",
    "model",
    "manufacturername",
    "owner",
    "operator",
    "registration",
]

cols = [c for c in CANDIDATE_COLS if c.lower() in available_cols]
select_cols = ",".join(cols)
print("Using columns:", cols)

def fetch_aircraft_meta(icao24_values, chunk_size=800, sleep_s=0.4, max_retries=3) -> pd.DataFrame:
    out = []
    for i in tqdm(range(0, len(icao24_values), chunk_size)):
        chunk = icao24_values[i:i+chunk_size]
        in_list = ",".join([f"'{x}'" for x in chunk])
        q = f"SELECT {select_cols} FROM {AIRCRAFT_TABLE} WHERE icao24 IN ({in_list})"

        for attempt in range(max_retries):
            try:
                df = pd.read_sql(q, conn)
                out.append(df)
                break
            except Exception:
                if attempt == max_retries - 1:
                    raise
                time.sleep(sleep_s * (attempt + 1))

        time.sleep(sleep_s)

    if not out:
        return pd.DataFrame(columns=cols)
    return pd.concat(out, ignore_index=True).drop_duplicates()

aircraft = fetch_aircraft_meta(icao24_list)
print("Fetched rows:", len(aircraft))
print("Coverage (icao24 found):", aircraft["icao24"].nunique(), "/", len(icao24_list))
aircraft.head()


C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\1585428627.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SHOW TABLES FROM minio.osky", conn)
C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\1585428627.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"DESCRIBE {table}", conn)


Using AIRCRAFT_TABLE: minio.osky.acas_data4
Using columns: ['icao24']


  0%|          | 0/9 [00:00<?, ?it/s]C:\Users\HiWi\AppData\Local\Temp\ipykernel_4816\1585428627.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
  0%|          | 0/9 [00:23<?, ?it/s]


KeyboardInterrupt: 

## 6) Save for later joins in preprocessing


In [ ]:
OUT_DIR.mkdir(parents=True, exist_ok=True)
aircraft.to_parquet(OUT_PATH, index=False)
print("Saved:", OUT_PATH)


## Next step
In `03_data_preprocessing.ipynb`, join this table back using `icao24` (directly or extracted from `flight_id`) to attach aircraft type metadata and enable Schmidt-Appleman (formation) + ISSR (persistence).
